# Как работать с MiniO вручную

In [3]:
from minio import Minio
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

access_key = os.environ.get("MINIO_ACCESS_KEY")
secret_key = os.environ.get("MINIO_SECRET_KEY")

client = Minio(
    "s3.pyrogn.ru",
    access_key=access_key,
    secret_key=secret_key,
    secure=True,
)

BUCKET_NAME = "tasksai"


In [4]:
[obj.object_name for obj in client.list_objects("tasksai")]

['.gitkeep',
 'tickets_src.csv',
 'custom_data/',
 'dataset_tickets_timeseries/',
 'nlp/',
 'tickets_daily/']

In [11]:
client.list_buckets()

[Bucket('tasksai'),
 Bucket('tasksai-backup-20250112224354'),
 Bucket('tasksai-backup-20250112224924'),
 Bucket('tasksai-backup-20250112225031')]

In [ ]:
from minio import Minio
import os
from dotenv import load_dotenv, find_dotenv
import io

def test_minio_operations():
    client = Minio(
        "s3.pyrogn.ru",
        access_key=os.environ.get("MINIO_ACCESS_KEY"),
        secret_key=os.environ.get("MINIO_SECRET_KEY"),
        secure=True,
    )

    bucket_name = "tasksai"

    # Test 1: List buckets
    print("Test 1: Listing buckets")
    buckets = client.list_buckets()
    print([b.name for b in buckets])

    # Test 2: Create bucket if not exists
    print("\nTest 2: Creating bucket")
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
        print(f"Created bucket: {bucket_name}")
    else:
        print(f"Bucket {bucket_name} already exists")

    # Test 3: Upload small test file
    print("\nTest 3: Uploading test file")
    test_data = io.BytesIO(b"Hello, MinIO!")
    try:
        client.put_object(
            bucket_name,
            "test.txt",
            test_data,
            length=len(test_data.getvalue())
        )
        print("Upload successful")
    except Exception as e:
        print(f"Upload failed: {e}")

    # Test 4: List objects
    print("\nTest 4: Listing objects")
    try:
        objects = list(client.list_objects(bucket_name))
        print([obj.object_name for obj in objects])
    except Exception as e:
        print(f"List failed: {e}")

    # Test 5: Download test file
    print("\nTest 5: Downloading test file")
    try:
        data = client.get_object(bucket_name, "test.txt")
        print(f"Downloaded content: {data.read().decode()}")
    except Exception as e:
        print(f"Download failed: {e}")

if __name__ == "__main__":
    test_minio_operations()

Test 1: Listing buckets
['tasksai', 'tasksai-backup-20250112224354', 'tasksai-backup-20250112224924', 'tasksai-backup-20250112225031']

Test 2: Creating bucket
Bucket tasksai already exists

Test 3: Uploading test file
Upload successful

Test 4: Listing objects
['.gitkeep', 'test.txt', 'tickets_src.csv', 'custom_data/', 'dataset_tickets_timeseries/', 'nlp/', 'tickets_daily/']

Test 5: Downloading test file
Downloaded content: Hello, MinIO!


In [ ]:
from minio import Minio
import os
from dotenv import load_dotenv

load_dotenv()

def test_minio_access():
    client = Minio(
        "s3.pyrogn.ru",
        access_key=os.getenv("MINIO_ACCESS_KEY"),
        secret_key=os.getenv("MINIO_SECRET_KEY"),
        secure=True
    )

    # Test bucket listing
    print("Listing buckets:")
    buckets = client.list_buckets()
    for bucket in buckets:
        print(f"- {bucket.name}")

    bucket_name = "tasksai"

    # Test object listing
    print("\nListing objects in tasksai:")
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        print(f"- {obj.object_name} (size: {obj.size} bytes)")
        # Try to get object stats
        try:
            stat = client.stat_object(bucket_name, obj.object_name)
            print(f"  Can access: Yes")
        except Exception as e:
            print(f"  Can access: No - {str(e)}")


test_minio_access()

Listing buckets:
- tasksai
- tasksai-backup-20250112224354
- tasksai-backup-20250112224924
- tasksai-backup-20250112225031

Listing objects in tasksai:
- .gitkeep (size: 0 bytes)
  Can access: Yes
- custom_data/tree_proper.csv (size: 61728 bytes)
  Can access: No - S3 operation failed; code: AccessDenied, message: Access denied, resource: /tasksai/custom_data/tree_proper.csv, request_id: 181A0B1DAD52B714, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: tasksai, object_name: custom_data/tree_proper.csv
- custom_data/tree_proper_slice.csv (size: 61407 bytes)
  Can access: No - S3 operation failed; code: AccessDenied, message: Access denied, resource: /tasksai/custom_data/tree_proper_slice.csv, request_id: 181A0B1DB3E904A5, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: tasksai, object_name: custom_data/tree_proper_slice.csv
- dataset_tickets_timeseries/all_tickets.tsv (size: 28973112 bytes)
  Can access: Yes